In [12]:
import torch
import torch.nn as nn
import pandas as pd

# define the model architecture
class IMUModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(IMUModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers)
        self.linear = nn.Linear(hidden_size, 1)

    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.linear(x)
        return x

# define the contrastive loss function
def contrastive_loss(y_true, y_pred, margin=1):
    return torch.mean(y_true * torch.square(y_pred) + (1 - y_true) * torch.square(torch.clamp(margin - y_pred, 0)))

# load the IMU data into a numpy array
imu_data = np.genfromtxt('C:/Users/Sandalu Karunasena/Desktop/IMU.txt', delimiter=',')

# convert the data to PyTorch tensors
imu_data = torch.from_numpy(imu_data)


# split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(imu_data, imu_data, test_size=0.2)

# create the model and optimizer
model = IMUModel(input_size=3, hidden_size=32, num_layers=2)
optimizer = torch.optim.Adam(model.parameters())

# train the model
for epoch in range(10):  # train for 10 epochs
    # Forward pass
    x = X_train.to(torch.float)
    y = y_train.to(torch.float)
    y_pred = model(x)

    # Compute the contrastive loss
    loss = contrastive_loss(y, y_pred)
    print(loss)
    # Backpropagate and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # evaluate the model on the test set
with torch.no_grad():
    x = X_test.to(torch.float)
    y = y_test.to(torch.float)

    y_pred = model(x)
    test_loss = contrastive_loss(y, y_pred)

print(f'Test loss: {test_loss:.4f}')

# compute the prediction labels based on the predicted distances
pred_labels = (y_pred < 0.5).long()

# compute the accuracy
accuracy = (pred_labels == y).float().mean()

print(f'Accuracy: {accuracy:.4f}')


tensor(0.7531, grad_fn=<MeanBackward0>)
tensor(0.7392, grad_fn=<MeanBackward0>)
tensor(0.7255, grad_fn=<MeanBackward0>)
tensor(0.7119, grad_fn=<MeanBackward0>)
tensor(0.6984, grad_fn=<MeanBackward0>)
tensor(0.6849, grad_fn=<MeanBackward0>)
tensor(0.6716, grad_fn=<MeanBackward0>)
tensor(0.6582, grad_fn=<MeanBackward0>)
tensor(0.6448, grad_fn=<MeanBackward0>)
tensor(0.6313, grad_fn=<MeanBackward0>)
Test loss: 0.6010
Accuracy: 0.1667
